# Clean Wiki

In [ ]:
import datasets
from datasets import load_dataset, load_from_disk
from tqdm.autonotebook import tqdm

In [20]:
wikipedia_dataset = load_dataset("wikipedia", "20220301.en", split="train")

Found cached dataset wikipedia (/home/eugene/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [21]:
wikipedia_dataset = wikipedia_dataset.remove_columns(
    [col for col in wikipedia_dataset.column_names if col != "text"]
)  # only keep the 'text' column

In [22]:
wikipedia_dataset = wikipedia_dataset.map(lambda x: {"len": len(x["text"])}, num_proc=24)

Map (num_proc=24):   0%|          | 0/6458670 [00:00<?, ? examples/s]

In [23]:
sum(wikipedia_dataset["len"])

19560538957

In [6]:
# wikipedia_dataset = wikipedia_dataset.select(range(10))

In [7]:
# !python -m spacy download en_core_web_sm

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [9]:
def process_see_also(sents):
    start_pos = -1
    end_pos = -1
    for i, s in enumerate(sents):
        if len(s) < 9 and "See also" in s:
            start_pos = i
            continue
        if start_pos > 0:
            if s[0] == " ":
                end_pos = i
            else:
                break
    if start_pos < 0:
        return sents
    if end_pos - start_pos < 1:
        return sents
    sents[start_pos] = sents[start_pos] + ": " + ", ".join(sents[start_pos+1:end_pos]) + "."
    sents = sents[: start_pos + 1]
    return sents

In [10]:
def remove_references(sents):
    start_pos = -1
    for i, s in enumerate(sents):
        if len(s) < 12 and 'References' in s:
            start_pos = i
            break
    return sents[:start_pos]

In [11]:
def filter_empty(sents, th=3):
    return [s for s in sents if len(s) > th]

In [12]:
def clean_wiki(examples):
    sents_merged = []
    for text in examples["text"]:
        paragraphs = text.split("\n")
        sents = [sent.text for p in paragraphs for sent in nlp(p).sents]
        sents = filter_empty(sents)
        sents = process_see_also(sents)
        sents = remove_references(sents)
        sents_merged.extend(sents)
    return {"text": sents_merged}

In [ ]:
wikipedia_dataset = wikipedia_dataset.map(
    lambda x: clean_wiki(x),
    batched=True,
    remove_columns=wikipedia_dataset.column_names
)

Map (num_proc=24):   0%|          | 0/6458670 [00:00<?, ? examples/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
wikipedia_dataset.save_to_disk("wikipedia_dataset_cleaned.hf")

# bookcorpusopen dataset cleaning

In [1]:
import datasets
from datasets import load_dataset, load_from_disk
from tqdm.autonotebook import tqdm

In [2]:
bookcorpus_dataset = load_dataset("bookcorpusopen", "plain_text", split="train")

Found cached dataset bookcorpusopen (/home/eugene/.cache/huggingface/datasets/bookcorpusopen/plain_text/1.0.0/98559c92eb612e150a676c5b5131f9f8f07d4cab88e7f3761fda266ad22ff2a7)


In [3]:
bookcorpus_dataset = bookcorpus_dataset.remove_columns(
    [col for col in bookcorpus_dataset.column_names if col != "text"]
)  # only keep the 'text' column

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
def filter_empty(sents, th=3):
    return [s for s in sents if len(s) > th]

In [6]:
def clean_bookcorpus(examples):
    sents_merged = []
    for text in examples["text"]:
        paragraphs = text.split("\n")
        sents = [sent.text for p in paragraphs for sent in nlp(p).sents]
        sents = filter_empty(sents)
        sents_merged.extend(sents)
    return {"text": sents_merged}

In [7]:
bookcorpus_dataset = bookcorpus_dataset.map(
    clean_bookcorpus,
    batched=True,
    remove_columns=bookcorpus_dataset.column_names,
    batch_size=1,
    num_proc=24
)

Map (num_proc=24):   0%|          | 0/17868 [00:00<?, ? examples/s]

In [8]:
bookcorpus_dataset.save_to_disk("bookcorpus_dataset_cleaned.hf")

Saving the dataset (0/14 shards):   0%|          | 0/97892049 [00:00<?, ? examples/s]

Notebook is based on https://huggingface.co/blog/how-to-train

## Load a dataset

In [1]:
import datasets
from datasets import load_dataset, load_from_disk
from tqdm.autonotebook import tqdm

In [2]:
wikipedia_dataset = load_from_disk("wikipedia_dataset_cleaned.hf")

In [3]:
bookcorpus_dataset = load_from_disk("bookcorpus_dataset_cleaned.hf")

In [4]:
# wikipedia_dataset = wikipedia_dataset.map(lambda x: {"len": len(x["text"])}, num_proc=24)
# sum(wikipedia_dataset["len"])

In [5]:
assert bookcorpus_dataset.features.type == wikipedia_dataset.features.type

In [6]:
raw_datasets = datasets.combine.concatenate_datasets([wikipedia_dataset, bookcorpus_dataset])

In [7]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", use_fast=True, add_prefix_space=True)

In [8]:
from tokenizers import normalizers
from tokenizers.normalizers import BertNormalizer

In [9]:
tokenizer.backend_tokenizer.normalizer = normalizers.Sequence([BertNormalizer()])

In [10]:
def get_training_corpus():
    return (
        raw_datasets[i : i + 1000]["text"]
        for i in tqdm(range(0, len(raw_datasets), 1000))
    )

In [11]:
tokenizer.tokenize("Hello world")

['Ġhello', 'Ġworld']

In [ ]:
training_corpus = get_training_corpus()
tokenizer = tokenizer.train_new_from_iterator(training_corpus, vocab_size=50000)

  0%|          | 0/303341 [00:00<?, ?it/s]

In [ ]:
tokenizer.tokenize("Hello world")

In [ ]:
tokenizer.save_pretrained("bergman-tokenizer")

In [ ]:
def tokenization(example):
    return tokenizer(example["text"])

raw_datasets = raw_datasets.map(tokenization, batched=True, num_proc=24)

In [ ]:
raw_datasets.save_to_disk("raw_dataset_tokenized.hf")

## 3. Train a language model from scratch

In [ ]:
# Check that we have a GPU
!nvidia-smi

In [1]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [2]:
import time

def is_gpu_free(th=1E+9):
    free, total = torch.cuda.mem_get_info()
    return total - free < th
    
while not(is_gpu_free()):
    time.sleep(60)

### We'll define the following config for the model

In [1]:
from bergman import BergmanConfig

# # Feb24_04-34-34_raven
# # Mar04_04-08-58_raven
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_matrix_heads=24,
#     num_hidden_layers=4,
#     type_vocab_size=1,
# )

# # Mar08_00-33-28_raven
# # 1035de06e
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=4,
#     type_vocab_size=1,
#     hidden_size=768,
# #     position_embedding_type="none",
#     matrix_norm_alg="-1",
#     matrix_dim=16,
#     num_matrix_heads=24,
#     vector_init_direction="one",
#     use_for_context=["lr", "rl"],
#     networks_for_heads=None,
#     matrix_norm_loss_type=None,
# )

# # Mar08_18-18-03_raven
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=4,
#     type_vocab_size=1,
#     hidden_size=768,
# #     position_embedding_type="none",
#     matrix_norm_alg="-1",
#     matrix_dim=16,
#     num_matrix_heads=24,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl"],
#     networks_for_heads=None,
#     matrix_norm_loss_type=None,
# )

# # Mar09_14-45-05_raven
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=2,
#     type_vocab_size=1,
#     hidden_size=768,
#     position_embedding_type="none",
#     matrix_norm_alg="-1",
#     matrix_dim=16,
#     num_matrix_heads=24,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl"],
#     networks_for_heads=None,
#     matrix_norm_loss_type=None,
# )

# # Mar10_00-11-30_raven
# # 1035de06e
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=2,
#     type_vocab_size=1,
#     hidden_size=768,
#     position_embedding_type="none",
#     matrix_norm_alg="-1",
#     matrix_dim=16,
#     num_matrix_heads=16,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl", "local_r"],
#     networks_for_heads=None,
#     matrix_norm_loss_type=None,
# )

# # Mar15_22-56-53_raven
# # fb6bf8afb
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=2,
#     type_vocab_size=1,
#     hidden_size=768,
#     position_embedding_type="none",
#     matrix_norm_alg="-1",
#     matrix_dim=16,
#     num_matrix_heads=16,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl", "local_r"],
#     networks_for_heads=None,
#     matrix_encoder_two_layers=False,
#     #
#     matrix_norm_loss_type="MSE",
#     matrix_norm_loss_k=0.0,
#     matrix_unitary_loss=None,
#     matrix_unitary_loss_k = 0.0,
#     norm_vectors=False,
#     detach_norm_vectors=False,
# )

# # Mar17_14-38-04_raven
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=2,
#     type_vocab_size=1,
#     hidden_size=768,
#     position_embedding_type="none",
#     matrix_norm_alg=-1,
#     matrix_dim=8,
#     num_matrix_heads=16,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl", "local_r"],
#     networks_for_heads=None,
#     matrix_encoder_two_layers=False,
#     #
#     matrix_norm_loss_type="MSE",
#     matrix_norm_loss_k=0.0,
#     matrix_unitary_loss=None,
#     matrix_unitary_loss_k = 0.0,
#     norm_vectors=False,
#     detach_norm_vectors=False,
#     complex_matrix=True,
#     complex_matrix_abs=False,
# )

# # Mar20_15-00-34_raven
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=2,
#     type_vocab_size=1,
#     hidden_size=768,
#     position_embedding_type="none",
#     matrix_norm_alg=None,
#     matrix_dim=16,
#     num_matrix_heads=16,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl", "local_r"],
#     networks_for_heads=None,
#     matrix_encoder_two_layers=False,
#     #
#     matrix_norm_loss_type="MSE",
#     matrix_norm_loss_k=0.0,
#     matrix_unitary_loss=None,
#     matrix_unitary_loss_k = 0.0,
#     norm_vectors=True,
#     complex_matrix=True,
#     complex_matrix_abs=True,
# )

# # Mar22_01-26-24_raven
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=2,
#     type_vocab_size=1,
#     hidden_size=768,
#     position_embedding_type="none",
#     matrix_norm_alg=None,
#     matrix_dim=16,
#     num_matrix_heads=16,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl", "local_r"],
#     networks_for_heads=None,
#     matrix_encoder_two_layers=False,
#     #
#     matrix_norm_loss_type="MSE",
#     matrix_norm_loss_k=0.0,
#     matrix_unitary_loss=None,
#     matrix_unitary_loss_k = 0.0,
#     norm_vectors=True,
#     complex_matrix=False,
#     complex_matrix_abs=True,
# )

# # Mar22_16-53-29_raven
# config = BergmanConfig(
#     vocab_size=52_000,
#     max_position_embeddings=514,
#     num_hidden_layers=2,
#     type_vocab_size=1,
#     hidden_size=768,
#     position_embedding_type="none",
#     matrix_norm_alg=None,
#     matrix_dim=32,
#     num_matrix_heads=8,
#     vector_init_direction="one",
#     use_for_context=["lr_excl", "rl_excl", "local_r"],
#     networks_for_heads=None,
#     matrix_encoder_two_layers=False,
#     #
#     matrix_norm_loss_type="MSE",
#     matrix_norm_loss_k=0.0,
#     matrix_unitary_loss=None,
#     matrix_unitary_loss_k = 0.0,
#     norm_vectors=True,
#     complex_matrix=False,
#     complex_matrix_abs=True,
# )

#
config = BergmanConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_hidden_layers=4,
    type_vocab_size=1,
    hidden_size=768,
    position_embedding_type="none",
    matrix_norm_alg=None,
    matrix_dim=4,
    num_matrix_heads=64,
    vector_init_direction="one",
    use_for_context=["lr_excl", "rl_excl", "local_r"],
    networks_for_heads=None,
    matrix_encoder_two_layers=False,
    #
    matrix_norm_loss_type=None,
    matrix_norm_loss_k=0.0,
    matrix_unitary_loss=None,
    matrix_unitary_loss_k = 0.0,
    norm_vectors=True,
    complex_matrix=True,
    complex_matrix_abs=True,
)

Now let's re-create our tokenizer in transformers

In [2]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./EsperBergman", max_len=512)

Finally let's initialize our model.

**Important:**

As we are training from scratch, we only initialize from a config, not from an existing pretrained model or checkpoint.

In [3]:
from bergman import BergmanForMaskedLM

model = BergmanForMaskedLM(config=config)

In [4]:
model.num_parameters()
# => 84 million parameters

68541216

In [5]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name, param.size(), param.numel())

### Now let's build our training Dataset

We'll build our dataset by applying our tokenizer to our text file.

Here, as we only have one text file, we don't even need to customize our `Dataset`. We'll just use the `LineByLineDataset` out-of-the-box.

In [6]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./oscar.eo.txt",
    block_size=128,
)

/home/eugene/Projects/matrix_network/venv/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 6min 43s, sys: 4.17 s, total: 6min 47s
Wall time: 43.3 s


Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

### Finally, we are all set to initialize our Trainer

In [8]:
from transformers import Trainer, TrainingArguments

In [9]:
training_args = TrainingArguments(
    output_dir="./EsperBergman",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=45,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=10,
    learning_rate=5E-5
)

In [10]:
from transformers.trainer import (
    MODEL_FOR_CAUSAL_LM_MAPPING_NAMES,
    is_torch_tpu_available,
)
import torch


class BergmanTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        metrics = outputs["metrics"] if isinstance(outputs, dict) else outputs[-1]
        self.metrics = {
            m: v if isinstance(v, float) else v.detach() for m, v in metrics.items()
        }

        if labels is not None:
            if (
                unwrap_model(model)._get_name()
                in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values()
            ):
                loss = self.label_smoother(outputs, labels, shift_labels=True)
            else:
                loss = self.label_smoother(outputs, labels)
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                raise ValueError(
                    "The model did not return a loss from the inputs, only the following keys: "
                    f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                )
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss

    def _maybe_log_save_evaluate(
        self, tr_loss, model, trial, epoch, ignore_keys_for_eval
    ):
        if not hasattr(self, "metrics_acc"):
            self.metrics_acc: Dict[str, torch.Tensor] = {}

        for m, v in self.metrics.items():
            if v is None:
                continue
            if m not in self.metrics_acc:
                self.metrics_acc[m] = torch.tensor(0.0).to(model.device)
            self.metrics_acc[m] += v

        if self.control.should_log:
            if is_torch_tpu_available():
                xm.mark_step()

            metrics = {
                m: self._nested_gather(v).mean().item()
                for m, v in self.metrics_acc.items()
            }
            # reset counters
            self.metrics_acc = {}

            logs = {
                m: round(
                    v / (self.state.global_step - self._globalstep_last_logged),
                    4,
                )
                for m, v in metrics.items()
            }

            # all_gather + mean() to get average loss over all processes
            tr_loss_scalar = self._nested_gather(tr_loss).mean().item()

            # reset tr_loss to zero
            tr_loss -= tr_loss

            logs["loss"] = round(
                tr_loss_scalar
                / (self.state.global_step - self._globalstep_last_logged),
                4,
            )
            logs["learning_rate"] = self._get_learning_rate()

            self._total_loss_scalar += tr_loss_scalar
            self._globalstep_last_logged = self.state.global_step
            self.store_flos()

            self.log(logs)

        metrics = None
        if self.control.should_evaluate:
            if isinstance(self.eval_dataset, dict):
                for eval_dataset_name, eval_dataset in self.eval_dataset.items():
                    metrics = self.evaluate(
                        eval_dataset=eval_dataset,
                        ignore_keys=ignore_keys_for_eval,
                        metric_key_prefix=f"eval_{eval_dataset_name}",
                    )
            else:
                metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)
            self._report_to_hp_search(trial, self.state.global_step, metrics)

        if self.control.should_save:
            self._save_checkpoint(model, trial, metrics=metrics)
            self.control = self.callback_handler.on_save(
                self.args, self.state, self.control
            )

In [11]:
trainer = BergmanTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


### Start training

In [ ]:
%%time
# with torch.autograd.detect_anomaly(True):
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/home/eugene/Projects/matrix_network/venv/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a

Step,Training Loss
10,10.424200
20,9.872900
30,9.481000
40,9.143300
50,8.928000
60,8.626600
70,8.537800
80,8.426400
90,8.255700
100,8.070900


#### 🎉 Save final model (+ tokenizer + config) to disk

In [ ]:
trainer.save_model("./EsperBergman")

## 4. Check that the LM actually trained

Aside from looking at the training and eval losses going down, the easiest way to check whether our language model is learning anything interesting is via the `FillMaskPipeline`.

Pipelines are simple wrappers around tokenizers and models, and the 'fill-mask' one will let you input a sequence containing a masked token (here, `<mask>`) and return a list of the most probable filled sequences, with their probabilities.



In [ ]:
model = model.to("cpu")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

In [ ]:
# The sun <mask>.
# =>

fill_mask("La suno <mask>.")

In [ ]:
fill_mask("Jen la komenco de bela <mask>.")

# Save graph

In [ ]:
torch.onnx.export(model, torch.LongTensor([[0,0,0,0,0]]), 'Bergman.onnx')

Ok, simple syntax/grammar works. Let’s try a slightly more interesting prompt:



## 5. Share your model 🎉

Finally, when you have a nice model, please think about sharing it with the community:

- upload your model using the CLI: `transformers-cli upload`
- write a README.md model card and add it to the repository under `model_cards/`. Your model card should ideally include:
    - a model description,
    - training params (dataset, preprocessing, hyperparameters), 
    - evaluation results,
    - intended uses & limitations
    - whatever else is helpful! 🤓

### **TADA!**

➡️ Your model has a page on http://huggingface.co/models and everyone can load it using `AutoModel.from_pretrained("username/model_name")`.

[![tb](https://huggingface.co/blog/assets/01_how-to-train/model_page.png)](https://huggingface.co/julien-c/EsperBERTo-small)


If you want to take a look at models in different languages, check https://huggingface.co/models

[![all models](https://huggingface.co/front/thumbnails/models.png)](https://huggingface.co/models)
